# Optimizing an ML Pipeline in Azure

In [1]:
import azureml.core
print(azureml.core.VERSION)

1.49.0


In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-235129
Azure region: eastus2
Subscription id: d7f39349-a66b-446e-aba6-0053c2cf1c11
Resource group: aml-quickstarts-235129


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "mxpipeline"

# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print(f"Found existing compute target: {compute_target}")
except:
    print(f"Creating a new compute target...")
    compute_cnfg = AmlCompute.provisioning_configuration(
        vm_size="Standard_D2_V2",
        min_nodes=0,
        max_nodes=4
    )

    compute_target = ComputeTarget.create(
        ws,
        cluster_name, 
        compute_cnfg
    )

    compute_target.wait_for_completion(
        show_output=True,
        min_node_count=None, 
        timeout_in_minutes=60
    )

# print message if ready
print(compute_target.get_status().serialize())

Creating a new compute target...
InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2023-06-06T00:13:53.053000+00:00', 'errors': None, 'creationTime': '2023-06-06T00:13:48.625579+00:00', 'modifiedTime': '2023-06-06T00:13:55.235861+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        "--C": uniform(0.0, 5.0),
        "--max_iter": choice(0, 50, 100, 150, 200),
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
compute_target_hdr = ws.compute_targets["mxpipeline"]
print(compute_target_hdr.get_status().serialize())

"""
# Please note: this does not do the job and creates aborts in the cells below. Please update your instructions.
src = ScriptRunConfig(
    source_directory=".",
    script="train.py",
    compute_target=compute_target_hdr, 
    environment=sklearn_env,
)
"""
est = SKLearn(source_directory=".", compute_target=compute_target_hdr, entry_script="train.py")


# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    estimator = est, 
    hyperparameter_sampling= ps, 
    policy= policy, 
    primary_metric_name='Accuracy', 
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
    max_total_runs=10,
    max_concurrent_runs=4,
)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2023-06-06T00:13:53.053000+00:00', 'errors': None, 'creationTime': '2023-06-06T00:13:48.625579+00:00', 'modifiedTime': '2023-06-06T00:13:55.235861+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [6]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hdr = exp.submit(config=hyperdrive_config)
RunDetails(hdr).show()
hdr.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_ce2bdda6-8cc6-438e-af73-6c5f2e0c15c6
Web View: https://ml.azure.com/runs/HD_ce2bdda6-8cc6-438e-af73-6c5f2e0c15c6?wsid=/subscriptions/d7f39349-a66b-446e-aba6-0053c2cf1c11/resourcegroups/aml-quickstarts-235129/workspaces/quick-starts-ws-235129&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-06-06T00:14:52.644619][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-06-06T00:14:52.9777680Z][SCHEDULER][INFO]Scheduling job, id='HD_ce2bdda6-8cc6-438e-af73-6c5f2e0c15c6_0' 
[2023-06-06T00:14:53.1206381Z][SCHEDULER][INFO]Scheduling job, id='HD_ce2bdda6-8cc6-438e-af73-6c5f2e0c15c6_1' 
[2023-06-06T00:14:53.2453249Z][SCHEDULER][INFO]Scheduling job, id='HD_ce2bdda6-8cc6-438e-af73-6c5f2e0c15c6_2' 
[2023-06-06T00:14:53.314136][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2023-06-06T00:14:53.3581359Z][SCHEDULER][INFO]Scheduling job, id='HD_ce2bdda6-8cc6-438e-af73-6c5f2e0c15

{'runId': 'HD_ce2bdda6-8cc6-438e-af73-6c5f2e0c15c6',
 'target': 'mxpipeline',
 'status': 'Completed',
 'startTimeUtc': '2023-06-06T00:14:52.11075Z',
 'endTimeUtc': '2023-06-06T00:22:23.617209Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '44817fb0-fd16-40f5-b350-cc618ea05050',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1035-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.49.0',
  'space_size': 'infinite_space_size',
  'score': '0.9139354212187424',
  'best_child_run_id': 'HD_ce2bdda6-8cc6-438e-af73-6c5f2e0c15c6_0',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_ce2bdda6-8cc6-438e-af73-6c5f2e0c15c6_0'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues

In [7]:
import joblib

# Get your best run and save the model from that run.
best_run = hdr.get_best_run_by_primary_metric()
best_run_results = best_run.get_metrics()

print(f"Best run: {best_run.id}")
print(f"Accuracy best run: {best_run_results['Accuracy']}")

Best run: HD_ce2bdda6-8cc6-438e-af73-6c5f2e0c15c6_0
Accuracy best run: 0.9139354212187424


In [8]:
best_run.get_file_names()

['outputs/model.joblib',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/metrics_capability/metrics-capability.log',
 'system_logs/snapshot_capability/snapshot-capability.log',
 'user_logs/std_log.txt']

In [9]:
best_run.download_files()

In [10]:
model = best_run.register_model(
    model_name="model.joblib",
    model_path="outputs/model.joblib"
)

In [11]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=url)

In [12]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

print(x.head())
print(y.head())

   age  marital  default  housing  loan  month  day_of_week  duration  \
0   57        1        0        0     1      5            1       371   
1   55        1        0        1     0      5            4       285   
2   33        1        0        0     0      5            5        52   
3   36        1        0        0     0      6            5       355   
4   27        1        0        1     0      7            5       189   

   campaign  pdays  ...  contact_cellular  contact_telephone  \
0         1    999  ...                 1                  0   
1         2    999  ...                 0                  1   
2         1    999  ...                 1                  0   
3         4    999  ...                 0                  1   
4         2    999  ...                 1                  0   

   education_basic.4y  education_basic.6y  education_basic.9y  \
0                   0                   0                   0   
1                   0                   0     

In [13]:
import pandas as pd

y_df = y.to_frame()
y_df.head()

,y
0,0
1,0
2,0
3,0
4,0


In [14]:
train_ds = pd.concat([x, y_df["y"]], axis=1)
train_ds.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
0,57,1,0,0,1,5,1,371,1,999,...,0,0,0,0,1,0,0,0,0,0
1,55,1,0,1,0,5,4,285,2,999,...,1,0,0,0,0,0,0,0,1,0
2,33,1,0,0,0,5,5,52,1,999,...,0,0,0,1,0,0,0,0,0,0
3,36,1,0,0,0,6,5,355,4,999,...,1,0,0,0,1,0,0,0,0,0
4,27,1,0,1,0,7,5,189,2,999,...,0,0,0,0,1,0,0,0,0,0


In [15]:
## convert from pandas dataframe to  Tabulardata
train_ds.to_csv("training/traindata.csv")
datastore =  ws.get_default_datastore()
datastore.upload(src_dir="./training",  target_path="data")

Uploading an estimated of 1 files
Uploading ./training/traindata.csv
Uploaded ./training/traindata.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_e44e163c38a54464a2a353622b2082f3

In [16]:
train_data = TabularDatasetFactory.from_delimited_files(
    path = [(datastore, ("data/traindata.csv"))]
)

In [17]:
from azureml.train.automl import AutoMLConfig

In [18]:
# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=train_data,
    label_column_name="y",
    n_cross_validations=5,
    compute_target=compute_target)

In [19]:
# Submit your automl run
run = exp.submit(config=automl_config, show_output=True)
RunDetails(run).show()
run.wait_for_completion(show_output=True)

Submitting remote run.
No run_configuration provided, running on mxpipeline with default configuration
Running on remote compute: mxpipeline


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_38ac3d56-0b17-41a3-9f46-161503a1963f,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+-------------------------------------

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_38ac3d56-0b17-41a3-9f46-161503a1963f,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Number of samples in the training data|
+==============================+================================+======================================+
|3692                          |1                               |32950                                 |
+------------------------------+--------------------------------+----

{'runId': 'AutoML_38ac3d56-0b17-41a3-9f46-161503a1963f',
 'target': 'mxpipeline',
 'status': 'Completed',
 'startTimeUtc': '2023-06-06T00:23:41.753975Z',
 'endTimeUtc': '2023-06-06T01:03:22.219444Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'mxpipeline',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"d7f39349-a66b-446e-aba6-0053c2cf1c11","resource_group":"aml-quickstarts-235129","workspace_name":"quick-starts-ws-235129","region":"eastus2","compute_target":"mxpipeline","spark_serv

In [26]:
!pip install azureml.core~=1.50.0

     |████████████████████████████████| 3.3 MB 4.9 MB/s eta 0:00:01
ERROR: azureml-widgets 1.49.0 has requirement azureml-core~=1.49.0, but you'll have azureml-core 1.50.0 which is incompatible.
ERROR: azureml-train-core 1.49.0 has requirement azureml-core~=1.49.0, but you'll have azureml-core 1.50.0 which is incompatible.
ERROR: azureml-train-automl-runtime 1.49.0 has requirement azureml-core~=1.49.0, but you'll have azureml-core 1.50.0 which is incompatible.
ERROR: azureml-train-automl-client 1.49.0 has requirement azureml-core~=1.49.0, but you'll have azureml-core 1.50.0 which is incompatible.
ERROR: azureml-tensorboard 1.49.0 has requirement azureml-core~=1.49.0, but you'll have azureml-core 1.50.0 which is incompatible.
ERROR: azureml-telemetry 1.49.0 has requirement azureml-core~=1.49.0, but you'll have azureml-core 1.50.0 which is incompatible.
ERROR: azureml-sdk 1.49.0 has requirement azureml-core~=1.49.0, but you'll have azureml-core 1.50.0 which is incompatible.
ERROR: azurem

In [ ]:
# Retrieve and save your best automl model.
best_aml_run, best_aml_model  =  run.get_output()

In [ ]:
best_aml_run.register_model(
    model_name="model.pkl", 
    model_path= "./outputs"
)
print(best_aml_model.final_estimator)

In [ ]:
compute_target.delete()